In [8]:
import pandas as pd
import pandas_datareader as pdr
from findatapy.market import Market, MarketDataGenerator, MarketDataRequest
from findatapy.util import SwimPool; SwimPool()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

I just realized I haven't really been clear with what I am attempting to accomplish with providing robust resampling of prices. I think it would help me if I lay out what some stuff.

Consider the following dataframe from Yahoo finance.


In [13]:
df = pdr.get_data_yahoo(['FB', 'TSLA'], '11/01/19',  interval = 'd')
df.columns
df

MultiIndex(levels=[['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'], ['FB', 'TSLA']],
           codes=[[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5], [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]],
           names=['Attributes', 'Symbols'])

Attributes        High                     Low                    Open  \
Symbols             FB        TSLA          FB        TSLA          FB   
Date                                                                     
2019-11-01  194.110001  316.480011  189.910004  309.799988  192.850006   
2019-11-04  197.369995  321.940002  193.809998  309.260010  194.550003   
2019-11-05  195.750000  323.510010  193.600006  316.119995  195.369995   
2019-11-06  194.369995  326.720001  191.350006  314.500000  194.029999   
2019-11-07  193.440002  341.500000  189.470001  328.019989  191.910004   
2019-11-08  192.339996  337.459991  189.699997  332.500000  190.000000   
2019-11-11  190.080002  349.190002  188.539993  342.000000  189.929993   
2019-11-12  195.059998  350.369995  189.740005  344.040009  190.000000   
2019-11-13  195.699997  356.329987  192.740005  345.179993  194.699997   
2019-11-14  194.029999  353.839996  191.449997  342.910004  192.929993   
2019-11-15  195.300003  352.799988  193.380005  348.359985  194.259995   
2019-11-18  198.630005  353.149994  193.050003  346.100006  194.559998   
2019-11-19  200.000000  359.989990  196.860001  347.799988  197.399994   
2019-11-20  199.589996  361.200012  195.429993  349.570007  198.580002   
2019-11-21  199.089996  360.839996  196.860001  354.000000  197.419998   
2019-11-22  199.300003  341.000000  197.619995  330.000000  198.380005   
2019-11-25  200.970001  344.570007  199.250000  334.459991  199.520004   
2019-11-26  200.149994  335.500000  198.039993  327.100006  200.000000   
2019-11-27  203.139999  333.915894  199.419998  328.570007  199.899994   

Attributes                   Close                Volume            \
Symbols           TSLA          FB        TSLA        FB      TSLA   
Date                                                                 
2019-11-01  316.320007  193.619995  313.309998  21711800   6383900   
2019-11-04  314.799988  194.720001  317.470001  16371300   8787000   
2019-11-05  319.619995  194.320007  317.220001   9942000   6943400   
2019-11-06  318.000000  191.550003  326.579987  10973000   7940900   
2019-11-07  329.140015  190.419998  335.540009  13473000  14467300   
2019-11-08  334.500000  190.839996  337.140015  10760800   6069200   
2019-11-11  343.950012  189.610001  345.089996   8631200   9986700   
2019-11-12  346.899994  194.470001  349.929993  17615500   7359400   
2019-11-13  355.000000  193.190002  346.109985  10860700   8420100   
2019-11-14  346.109985  193.149994  349.350006   9040500   6464900   
2019-11-15  350.640015  195.100006  352.170013  11524300   4809000   
2019-11-18  352.920013  197.399994  349.989990  16167200   4400400   
2019-11-19  351.750000  199.320007  359.519989  19056800   7724800   
2019-11-20  360.000000  197.509995  352.220001  12355400   6725100   
2019-11-21  354.510010  197.929993  354.829987  12131000   6110000   
2019-11-22  340.160004  198.820007  333.040009   9959800  16870600   
2019-11-25  344.320007  199.789993  336.339996  15272300  12339500   
2019-11-26  335.269989  198.970001  328.920013  11735500   7947400   
2019-11-27  331.119995  202.000000  331.290009  12234087   5455268   

Attributes   Adj Close              
Symbols             FB        TSLA  
Date                                
2019-11-01  193.619995  313.309998  
2019-11-04  194.720001  317.470001  
2019-11-05  194.320007  317.220001  
2019-11-06  191.550003  326.579987  
2019-11-07  190.419998  335.540009  
2019-11-08  190.839996  337.140015  
2019-11-11  189.610001  345.089996  
2019-11-12  194.470001  349.929993  
2019-11-13  193.190002  346.109985  
2019-11-14  193.149994  349.350006  
2019-11-15  195.100006  352.170013  
2019-11-18  197.399994  349.989990  
2019-11-19  199.320007  359.519989  
2019-11-20  197.509995  352.220001  
2019-11-21  197.929993  354.829987  
2019-11-22  198.820007  333.040009  
2019-11-25  199.789993  336.339996  
2019-11-26  198.970001  328.920013  
2019-11-27  202.000000  331.290009

## Above is dataframe of daily prices for Facebook and Tesla across 6 different fields. Now compare the above with the following:

In [12]:
df_weekly = pdr.get_data_yahoo(['FB', 'TSLA'], '11/01/19',  interval = 'w')
df_weekly

Attributes        High                     Low                    Open  \
Symbols             FB        TSLA          FB        TSLA          FB   
Date                                                                     
2019-10-28  194.110001  316.480011  189.910004  309.799988  192.850006   
2019-11-04  197.369995  341.500000  189.470001  309.260010  194.550003   
2019-11-11  195.699997  356.329987  188.539993  342.000000  189.929993   
2019-11-18  200.000000  361.200012  193.050003  330.000000  194.559998   
2019-11-25  200.970001  344.570007  198.039993  327.100006  199.520004   
2019-11-27  203.139999  333.915894  199.419998  328.570007  199.899994   

Attributes                   Close                Volume            \
Symbols           TSLA          FB        TSLA        FB      TSLA   
Date                                                                 
2019-10-28  316.320007  193.619995  313.309998  21711800   6383900   
2019-11-04  314.799988  190.839996  337.140015  61520100  44207800   
2019-11-11  343.950012  195.100006  352.170013  57672200  37040100   
2019-11-18  352.920013  198.820007  333.040009  69670200  41830900   
2019-11-25  344.320007  198.970001  328.920013  27007800  20286900   
2019-11-27  331.119995  202.000000  331.290009  12234087   5455268   

Attributes   Adj Close              
Symbols             FB        TSLA  
Date                                
2019-10-28  193.619995  313.309998  
2019-11-04  190.839996  337.140015  
2019-11-11  195.100006  352.170013  
2019-11-18  198.820007  333.040009  
2019-11-25  198.970001  328.920013  
2019-11-27  202.000000  331.290009

By the way, we read the observation interval of weeks as the date when the week starts. This is why the datetime index starts at `2019-10-28` as that was the week in which `11/01/19` (the start of the data requested) fell on

## Aside from the indexes, what properties does this 2nd dataframe share with the 1st dataframe?

## Answer: They are both are drawn from the same data.

As the datavendors record the price movements of stock they accumulate a lot of data. Tracking 1 stock every second of the trading day means... (ok, so there are 6.5 hours in a trading day and there are 3600 seconds in an hour so that's 6.5 times 3600 = 23,400 seconds), the single stock would generate 23,400 datapoints. For a trading day, uping the frequency of observations increases the number of datapoints, lowering the observation frequency decreases the number of datapoints. But customers might not care how the stock was doing moment to moement, they might only care about how the stock did overall on a given day. That's why open, high, low, and close have become so common place. They are a cleaner description of the stock's overall movements during a period of time. With the open price datapoint we know the starting price of that period. With the high price datapoint we know the maximum at which it traded during that period. With the low price datapoint we know the minimum price it traded at during that period. And with the closing price datapoint we know how it ended. These 4 datapoints are rich in information and could easily be used as inputs in a portfolio optimzation. In all the work I've done I've always used adj closing prices but I've always wanted to see what changes when you change the input data. Building out a means of swiftly achieving this would be helpful.


Effectively I want to achieve the same thing above but doing it all here on spot without calling for any more downloads (note*). A bit of code would work out the proper results. A weekly resample of the daily data would produce the same results seen above. You can imagine the same for monthly (or, god forbid, yearly) data. This would also set the stage for analyzing the impact intervals have on the outcome of an optimization, and subsequently, its results) 


note* Consider that it might just be faster to download the other interval data... (if that's the case why am I doing this... I guess at this point it's to learn)